<h1><center>Laboratorio 5: Benchmark Bayesiano 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Benjamín Tejeda
- Nombre de alumno 2: Constanza Peña 


### **Link de repositorio de GitHub:** `https://github.com/constanzaps/laboratorios_LDS`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo bayesiano para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [2]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image
from functools import lru_cache

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_6o2td1zc54671.jpg?width=4000&format=pjpg&s=600bcf8560dff264f1cc7c0785ba5f6d529e0c28" width="10000">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema Bayesiano, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a las subfunciones ser ejecutadas (por linea de código, vean las clases). Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del Código a través de un perfilador.

In [6]:
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [4]:
%load_ext line_profiler

In [7]:
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-07 s

Total time: 19.1726 s
File: <ipython-input-2-d987b25ddeb0>
Function: praw_reddit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents

    
     1                                           def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     2         1         31.0     31.0      0.0    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     3         1          9.0      9.0      0.0                       client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     4         1          7.0      7.0      0.0                       password='ClasesMDS7202',
     5         1          7.0      7.0      0.0                       user_agent='Clases',
     6         1      38113.0  38113.0      0.0                       username='DocenciaDataScience', check_for_async=False)
     7         1        355.0    355.0      0.0    subreddit  = reddit.subreddit(nombre_subreddit)
     8                                           
     9         1         14.0     14.0      0.0    votes, post, url = {}, {}, {}
    10         1  190621337.0 190621337.0     99.4    top_submissions = list(subreddit.hot(limit = n_hot))
    11        20        346.0     17.3      0.0    for it, top_n in enumerate(range(50, len(top_submissions),50)):
    12        19       4016.0    211.4      0.0      top_n_submissions = top_submissions[:top_n]
    13        19       1390.0     73.2      0.0      upvotes, downvotes, url[it], post[it] = [], [], [], []
    14                                           
    15      9519      97343.0     10.2      0.1      for submission in top_n_submissions:
    16      9500      93638.0      9.9      0.0          try:
    17      9500     129453.0     13.6      0.1              ratio = submission.upvote_ratio
    18      9500     219512.0     23.1      0.1              ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    19      9500     105647.0     11.1      0.1              upvotes.append(ups)
    20      9500     123223.0     13.0      0.1              downvotes.append(ups - submission.score)
    21      9500     128541.0     13.5      0.1              post[it].append(submission.title)
    22      9500     141666.0     14.9      0.1              url[it].append(submission.url)
    23                                                   except Exception as e:
    24                                                       continue
    25        19      21795.0   1147.1      0.0      votes[it] = np.array([upvotes, downvotes]).T
    26         1         12.0     12.0      0.0    return votes, post, url

In [ ]:
%load_ext memory_profiler

In [ ]:
%%file reddit_module.py

import praw
import numpy as np
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)
  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []
    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [ ]:
from reddit_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

Filename: C:\Users\benja\reddit_module.py

Line #    Mem usage    Increment  Occurences   Line Contents
============================================================
     3    143.1 MiB    143.1 MiB           1   def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     4    143.1 MiB      0.0 MiB           1     reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     5    143.1 MiB      0.0 MiB           1                        client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     6    143.1 MiB      0.0 MiB           1                        password='ClasesMDS7202',
     7    143.1 MiB      0.0 MiB           1                        user_agent='Clases',
     8    143.1 MiB      0.0 MiB           1                        username='DocenciaDataScience', check_for_async=False)
     9    143.1 MiB      0.0 MiB           1     subreddit  = reddit.subreddit(nombre_subreddit)
    10    143.1 MiB      0.0 MiB           1     votes, post, url = {}, {}, {}
    11    155.3 MiB     12.2 MiB           1     top_submissions = list(subreddit.hot(limit = n_hot))
    12    155.3 MiB      0.0 MiB          20     for it, top_n in enumerate(range(50, len(top_submissions),50)):
    13    155.3 MiB      0.0 MiB          19       top_n_submissions = top_submissions[:top_n]
    14    155.3 MiB      0.0 MiB          19       upvotes, downvotes, url[it], post[it] = [], [], [], []
    15    155.3 MiB      0.0 MiB        9519       for submission in top_n_submissions:
    16    155.3 MiB      0.0 MiB        9500           try:
    17    155.3 MiB      0.0 MiB        9500               ratio = submission.upvote_ratio
    18    155.3 MiB      0.0 MiB        9500               ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    19    155.3 MiB      0.0 MiB        9500               upvotes.append(ups)
    20    155.3 MiB      0.0 MiB        9500               downvotes.append(ups - submission.score)
    21    155.3 MiB      0.0 MiB        9500               post[it].append(submission.title)
    22    155.3 MiB      0.0 MiB        9500               url[it].append(submission.url)
    23                                                 except Exception as e:
    24                                                     continue
    25    155.3 MiB      0.0 MiB          19       votes[it] = np.array([upvotes, downvotes]).T
    26    155.3 MiB      0.0 MiB           1     return votes, post, url

**Respuesta:**

`Respecto a los tiempos de ejecución, se puede observar que el mayor tiempo de ejecución en la función lo tiene la linea 10, que la importa los post más activos desde Reddit. 
Respecto a la memoria, la linea que hace mayor uso de memoria es la linea 3, que corresponde a la definición de la función pues el la parte que guarda, ¿¿¿¿PQ?!
`

## 1.2 Análisis de Tiempo con Cache

Sin duda, un factor clave en la mejora del tiempo de ejecución de una aplicación es el uso eficiente de la memoria. Por lo que es importante, que ustedes respondan las siguientes preguntas:
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder preguntas.
- [ ] Mejorar el código con cache.


**Respuestas Teóricas:**

1.    ¿Qué es la memoria cache y a que se refiere las siglas LRU?

La memoria caché es una memoria de rápido acceso, que puede estar ubicado en la memoria RAM, el disco o almacenada de manera remota. Con esto se almacena un conjunto de instrucciones que se utilizan con asiduidad para realizar sus tareas cotidianas de forma más rápida.

LRU significa least recently used. Esta es una política de descarte de caché, donde se descarta primero los elementos menos usados recientemente.

2.    ¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?

Las técnicas de caching implican aumentar el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente. Antes de usar este tipo de estrategias, se recomienda estudiar la factibilidad, teniendo en cuenta las politicas de almacenamiento y acceso de los resultados y su relación con el rendimiento del programa que se desea implementar.

3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Con esto, el ocupar caching implica un uso de memoria, lo cual nos quita espacio para otras funciones, sin embargo nos permite realizar este algoritmo de extración de forma más rápida.

In [8]:
%%timeit
a, b, c = praw_reddit()

RequestException: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /r/chile/hot?limit=1000&raw_json=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000261D78E13A0>, 'Connection to oauth.reddit.com timed out. (connect timeout=16)'))

In [10]:
@lru_cache(maxsize=1000)
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)
  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []
    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [ ]:
%%timeit
a, b, c = praw_reddit()

## 1.3 Obtención de Mean Posterior y Standard Error

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia Bayesiana. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$
$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [11]:
votes, post, url = praw_reddit()
votos =votes[0]
post =post[0]
urls=url[0]

In [12]:
def intervalos(votos_positivos, votos_negativos):
    intervalo_mu=[]
    intervalo_sigma=[]
    for i in range(0, len(votos_positivos)):
        a=votos_positivos[i]+1
        b=votos_negativos[i]+1
        mu=a/(a+b)
        sigma=1.65*(a*b/((a+b)**(2)*(a+b+1)))**(1/2)
        intervalo_mu.append(mu)
        intervalo_sigma.append(sigma)
    return intervalo_mu,intervalo_sigma

 
def intervalos_numpy(votos_positivos, votos_negativos):
    a=votos_positivos+1
    b=votos_negativos+1
    intervalo_sigma=1.65*(np.sqrt((a*b)/((a+b)**(2)*(a+b+1))))
    intervalo_mu=a/(a+b)
    return intervalo_mu,intervalo_sigma


In [ ]:
intervalos_numpy(pd.DataFrame(votos)[0],pd.DataFrame(votos)[1])

In [ ]:
intervalos(pd.DataFrame(votos)[0],pd.DataFrame(votos)[1])

## 1.4 Comparación de rendimiento

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [ ]:
from numba import jit

@jit(nopython=True)
def intervalos_JIT(votos_positivos, votos_negativos):
    intervalo_mu=[]
    intervalo_sigma=[]
    for i in range(0, len(votos_positivos)):
        a=votos_positivos[i]+1
        b=votos_negativos[i]+1
        mu=a/(a+b)
        sigma=1.65*(a*b/((a+b)**(2)*(a+b+1)))**(1/2)
        intervalo_mu.append(mu)
        intervalo_sigma.append(sigma)
    return intervalo_mu,intervalo_sigma

@jit(nopython=True)
def intervalos_numpy_JIT(votos_positivos, votos_negativos):
    a=votos_positivos+1
    b=votos_negativos+1
    intervalo_sigma=1.65*(np.sqrt((a*b)/((a+b)**(2)*(a+b+1))))
    intervalo_mu=a/(a+b)
    return intervalo_mu,intervalo_sigma

In [ ]:
import timeit
tiempo_int=[]
for i in range(0,len(votes)):
    x=timeit.timeit(lambda:intervalos(votes[i][:,0],votes[i][:,1]), number=100)
    tiempo_int.append(x)

tiempo_intnp=[]
for i in range(0,len(votes)):
    x=timeit.timeit(lambda:intervalos_numpy(votes[i][:,0],votes[i][:,1]), number=100)
    tiempo_intnp.append(x)


tiempo_intJIT=[]
for i in range(0,len(votes)):
    x=timeit.timeit(lambda:intervalos_JIT(votes[i][:,0],votes[i][:,1]), number=100)
    tiempo_intJIT.append(x)

tiempo_intJITnp=[]
for i in range(0,len(votes)):
    x=timeit.timeit(lambda:intervalos_numpy_JIT(votes[i][:,0],votes[i][:,1]), number=100)
    tiempo_intJITnp.append(x)
    

In [ ]:
tiempos =pd.DataFrame(np.array([[*range(0, len(votes))],tiempo_int,tiempo_intnp,tiempo_intJIT,tiempo_intJITnp]).T)
tiempos.columns=['Tiempos','Intervalos','Intervalos numpy','Intervalos JIT','Intervalos JIT numpy']
df=pd.melt(tiempos, id_vars=['Tiempos'], value_vars=['Intervalos','Intervalos numpy','Intervalos JIT','Intervalos JIT numpy'],
        var_name='Modo', value_name='Duracion')
df

In [ ]:
fig = px.line(df, x="Tiempos", y="Duracion", color='Modo')
fig.show()

Respecto a los desempeños, estos son acordes a los observados, donde al usar las funciones básicas de python demora más y al usar las funciones con el decorador jit demora menos.

Esto nos deja los tiempos de la siguiente manera:

*Intervalos>Intervalos_JIT>Intervalos_numpy>Intervalos_JIT_numpy*

Es curioso que Intervalos_numpy tarde menos que Intervalos_JIT, pero lo atribuimos a la poca eficiencia de usar for en cada elemento, usando numpy menos memoria. Además de ello se puede ver en el gráfico que las diferencias son bastante despreciables para todos menos Intervalos, donde intervalos es la forma más básica de programar.

## 1.5 Plot de Resultados Bayesianos

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [ ]:
votes, post, url = praw_reddit()


votos = votes[len(votes)-1]
posteo = post[len(post)-1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {'Votos (+)':votos[order[:20], 0], 'Votos (-)':votos[order[:20], 1] ,'Post':np.array(posteo)[order[:20]], 'url': np.array(url[len(url)-1])[order[:20]]}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

In [ ]:
data_dic.values()

In [ ]:
r_order = order[::-1][-20:]
data_dic = {'mean':posterior_mean[r_order], 'std_err':std_err[r_order], 'post':ordered_post[::-1]}
data_dic
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post",
                 error_x="std_err")
fig.show()

**Respuesta:** 
Se puede observar que las probabilidades, probabilidad de que sea bueno el post


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>